In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d puneet6060/intel-image-classification


cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
100% 346M/346M [00:19<00:00, 23.7MB/s]
100% 346M/346M [00:19<00:00, 18.3MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/intel-image-classification.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
import os
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
# from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab Notebooks/intel_images/

/content/drive/MyDrive/Colab Notebooks/intel_images


In [ ]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/seg_train/seg_train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/seg_test/seg_test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 148, 148, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 72, 72, 64)        256       
 chNormalization)                                                
                                                        

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
elst= callbacks.EarlyStopping(monitor = 'val_loss', patience=5, mode='min')
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss', mode = 'min')

In [ ]:
model.fit(train_ds,epochs=10,validation_data=validation_ds, callbacks=[elst,save_ck])

Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 1.5672 - accuracy: 0.5361

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


439/439 [==============================] - 31s 52ms/step - loss: 1.5672 - accuracy: 0.5361 - val_loss: 1.7904 - val_accuracy: 0.5053
Epoch 2/10
439/439 [==============================] - 17s 38ms/step - loss: 0.8450 - accuracy: 0.7061 - val_loss: 1.1164 - val_accuracy: 0.6310
Epoch 3/10
439/439 [==============================] - 19s 42ms/step - loss: 0.6555 - accuracy: 0.7711 - val_loss: 1.0853 - val_accuracy: 0.6930
Epoch 4/10
439/439 [==============================] - 17s 40ms/step - loss: 0.5433 - accuracy: 0.8103 - val_loss: 0.7587 - val_accuracy: 0.7317
Epoch 5/10
439/439 [==============================] - 16s 36ms/step - loss: 0.4870 - accuracy: 0.8306 - val_loss: 1.0312 - val_accuracy: 0.6653
Epoch 6/10
439/439 [==============================] - 18s 42ms/step - loss: 0.4124 - accuracy: 0.8585 - val_loss: 0.6456 - val_accuracy: 0.7853
Epoch 7/10
439/439 [==============================] - 17s 39ms/step - loss: 0.3648 - accuracy: 0.8737 - val_loss: 0.6304 - val_accuracy: 0.7940
Epo